In [ ]:
!pip install faiss-cpu
!pip install urllib
!pip install huggingface_hub
!pip install langchain
!pip install unstructured
!pip install --upgrade --quiet  text-generation
!pip install langchainhub
!pip install sentencepiece
!pip install langchain-community
!pip install tiktoken
!pip install sentence-transformers
!pip install python-telegram-bot==20.0a0

In [ ]:
from langchain.memory import ChatMessageHistory

In [ ]:
from telegram import Update
from telegram.ext import CommandHandler, ContextTypes, ApplicationBuilder
import logging
from urllib.request import urlopen
from bs4 import BeautifulSoup as Soup
import requests
import re
import json
from langchain_community.document_loaders import JSONLoader
from pathlib import Path
from pprint import pprint
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
import os
import getpass
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
import numpy as np
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
logging.basicConfig(
  format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
  level=logging.INFO)

In [ ]:
DATABASE = None

In [ ]:
inference_api_key = '' #Your HF API token here!

In [ ]:
async def start(update: Update, context: ContextTypes):
    await context.bot.send_message(chat_id=update.effective_chat.id,
                                 text="Привет, ёлки-иголки! Задай любой вопрос про вселенную Смешариков или любой другой! Ты можешь задать вопрос, используя волшебную команду /query, за которой следует твой вопрос!")

In [ ]:
loader = TextLoader("scenarios.json")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
model_name="gpt-4",
chunk_size=1400,
chunk_overlap=100)
chunks = text_splitter.split_documents(documents)
embeddings = SentenceTransformerEmbeddings(model_name='intfloat/multilingual-e5-large')
DATABASE = FAISS.from_documents(chunks, embeddings)

<ipython-input-8-2514f643c0df>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name='intfloat/multilingual-e5-large')
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a toke

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFaceHub(
          huggingfacehub_api_token = inference_api_key,
          repo_id= "HuggingFaceH4/zephyr-7b-beta",
          task="text-generation",
          model_kwargs={
              "max_new_tokens": 512,
              "top_k": 30,
              "temperature": 0.1,
              "repetition_penalty": 1.03,
          },
      )

In [ ]:
async def query(update: Update, context: ContextTypes):
    PROMPT_TEMPLATE = """
    Ты дружелюбный чат-бот, который отвечает на вопросы про вселенную Смешариков на русском языке и с опорой на предоставленный контекст:

    {context}. Ты должен знать, что Смешарики - это российский мультсериал про шарообразных зверьков, который был создан в 2003 году студией Петербург.

    ---

    Ответь на вопрос с учётом предоставленного контекста: {question}
    """

    results = DATABASE.similarity_search_with_relevance_scores(update.message.text, k=3, score_threshold=0.65)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=update.message.text)

    response_text = llm.predict(prompt)
    def extract_response(response_text):
        index = response_text.find("/assistant")
        if index != -1:
            return response_text[index + len("/assistant"):].strip()

        index1 = response_text.find("/assistant:")
        if index1 != -1:
            return response_text[index1 + len("/assistant:"):].strip()

        index2 = response_text.find("Ответ:")
        if index2 != -1:
            return response_text[index2 + len("Ответ:"):].strip()

        context_index = response_text.find("Ответь на вопрос с учётом предоставленного контекста:")
        if context_index != -1:
           return response_text[context_index + len("Ответь на вопрос с учётом предоставленного контекста:"):].strip()

    await context.bot.send_message(chat_id=update.effective_chat.id,
                                   text=extract_response(response_text))

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
if __name__ == "__main__":
    application = ApplicationBuilder().token(os.getenv('TELEGRAM_BOT_TOKEN')).build()
    application.add_handler(CommandHandler('start', start))
    application.add_handler(CommandHandler('query', query))
    application.run_polling()